```python -m spacy download en_core_web_sm```

https://stackabuse.com/python-for-nlp-vocabulary-and-phrase-matching-with-spacy/

In [1]:
import os, re, time, sys
import subprocess

import json #serialise
import requests
from gtts import gTTS

class Voices():
    '''vv'''
    voices = {'K' : 'en-GB_KateV3Voice',
              'M' : 'en-US_MichaelV3Voice',
              'KK' : 'en-US_KevinV3Voice',
              'LI' : 'zh-CNLiNaVoice',
              'O' : 'en-US_OliviaV3Voice',
              'R' : 'fr-FR_ReneeV3Voice'
             }
    
    def __init__(self):
        if not os.path.isfile('script.json'):
            self.inventory = {}
        else:
            self.inventory = json.load(open('script.json', 'r'))

    def string2fn(self, xx):
        '''ff'''
        '''hash function
        strip punctuation
        return first 3 words with sep=_'''

        words = re.sub(r'[^\w\s]', '', xx).lower().split() #strip punctuation - > lowercase
        #check and pad
        if len(words) < 3:
            words.extend(['blah']*3)
        return '_'.join(words[:3]) + '.mp3'

    def get_audio(self, to_say):

        actor, txt = to_say
        FN = self.string2fn(txt)
        print('Doing', FN)
        
        if actor in self.voices:
            url = 'https://text-to-speech-demo.ng.bluemix.net/api/v3/synthesize'
            params = {'text' : txt,
                      'voice' : self.voices[actor],
                      'download' : 'true',
                      'accept' : 'audio/mp3'
            }
         
            r = requests.get(url, params=params)

            with open('%s'%FN, 'wb') as FP:
                FP.write(r.content)

        else: #assume it's a language tag and ask google
            tts = gTTS(txt, lang=actor.lower())
            tts.save(FN)

    def add(self, txts):

        for tt in txts:
            actor, lines = tt
            FN = self.string2fn(lines)
            key = actor + FN
            if key in self.inventory and self.inventory[key] == lines:
                print('skipping', FN)
                continue
            self.get_audio(tt)
            time.sleep(20)

        with open('script.json', 'w') as FP:
            json.dump({actor + self.string2fn(lines) : lines for  actor,lines  in txts}, FP)
        print('DONE')
        
    def __repr__(self):
        return str('\n'.join(self.inventory.keys()))

In [2]:
speek = Voices()

In [3]:
speek.add([('K','Imagination is more important than knowledge.')] )

Doing imagination_is_more.mp3
DONE


In [7]:
spacy.load('en_core_web_sm')   

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

tt = qq[10]
doc = nlp(tt)

print(tt)
for token in doc:
    if token.pos_ not in ['NOUN']: continue
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Weakness of attitude becomes weakness of character.
Weakness weakness NOUN NN nsubj Xxxxx True False
attitude attitude NOUN NN pobj xxxx True False
weakness weakness NOUN NN attr xxxx True False
character character NOUN NN pobj xxxx True False


In [1]:
import pdfplumber
with pdfplumber.open("/home/macbuse/Documents/750-Famous-Quotes.pdf") as pdf:
    page  = pdf.pages[10]
    text = page.chars[10]
    print(text)

{'fontname': 'TimesNewRomanPSMT', 'adv': Decimal('0.333'), 'upright': True, 'x0': Decimal('151.666'), 'y0': Decimal('692.568'), 'x1': Decimal('155.662'), 'y1': Decimal('704.568'), 'width': Decimal('3.996'), 'height': Decimal('12.000'), 'size': Decimal('12.000'), 'object_type': 'char', 'page_number': 11, 'stroking_color': None, 'non_stroking_color': [0, 0, 0], 'text': 'f', 'top': Decimal('87.432'), 'bottom': Decimal('99.432'), 'doctop': Decimal('8007.432')}


'/home/macbuse/Documents/macbuse.github.io/PROG'

In [2]:
from PyPDF2 import PdfFileReader

pdf_document = "/home/macbuse/Documents/750-Famous-Quotes.pdf"
with open(pdf_document, "rb") as fp:
    pdf = PdfFileReader(fp)
    pdf.decrypt('')
    info = pdf.getDocumentInfo()
    pages = pdf.getNumPages()

    print (info)
    print ("number of pages: %i" % pages)
    
    data = []
    for k in range(2,53):
        page = pdf.getPage(k)
    
        data.append(page.extractText())



{'/ModDate': "D:20040626211801-04'00'", '/CreationDate': "D:20040626210341-04'00'", '/Title': 'Famous Motivational', '/Creator': 'Acrobat PDFMaker 6.0 for Word', '/Author': 'George', '/Producer': 'Acrobat Distiller 6.0 (Windows)', '/Company': 'AL', '/SourceModified': 'D:20040627010124'}
number of pages: 53


In [3]:
import re

pp = re.compile('([^\.]*?)(\d+) quotes')

quotes = ''.join([ txt.replace('\n','') for txt in data])

In [4]:
quotes[:100]

'Abraham Lincoln 34 quotes    1. My father taught me to work; he did not teach me to love it.  2. Com'

In [5]:
pp.findall(quotes)

blocks = pp.sub('\n', quotes).split('\n')

In [6]:
pq = re.compile('\d+\.([^\d]+)')

qq = [x.strip() for x in pq.findall(blocks[2])]

In [7]:
with open('einstein.txt','w') as fp:
    fp.write('\n'.join(qq))